# 连接milvus

In [20]:
import numpy as np
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
connections.connect("default", host="localhost", port="19530")
collection = Collection("qa1")

# 修改/创建索引

https://milvus.io/docs/build_index.md

- 如果索引已存在，想修改：

    - 如果连接已加载（loaded），先释放（release），然后删除原有索引（drop_index）

    - 创建新索引
    
- 如果初次创建索引

    - 直接创建

In [23]:
def modify_index(field_name="vector", index=None):
    if not index:
        index = {
            "index_type": "HNSW",
            "metric_type": "L2",
            "params": {
                "M": 8,  # 可以根据需要调整此值
                "efConstruction": 64,  # 可以根据需要调整此值
            },
        }
    if collection.has_index(field_name=field_name):
        # if str(utility.load_state("pdf_test")) == 'Loaded':
        #     collection.release()
        collection.release()
        collection.drop_index()

    collection.create_index(field_name, index)

modify_index(field_name="vector")

# load() 

In [24]:
collection.load()

# search

## 搜索语句向量化

In [25]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('../../model/m3e-base')

sentences = ['每刻报销的报销流程是什么？']
embeddings = model.encode(sentences)
embeddings

array([[ 5.13300359e-01,  4.73528415e-01,  4.25761670e-01,
        -4.80491519e-01, -6.89796984e-01, -6.04210198e-01,
        -8.97260547e-01, -5.98901808e-01, -6.43777728e-01,
        -1.37697768e+00,  6.51556909e-01,  1.71763003e-01,
         4.96303022e-01,  1.85414821e-01,  5.30326247e-01,
         4.24150944e-01,  8.23790252e-01,  1.03078198e+00,
        -5.20769060e-01, -3.11526954e-01, -9.17735547e-02,
        -1.79168686e-01, -1.16570020e+00,  6.46890461e-01,
        -4.83959824e-01, -5.21964371e-01, -5.56099176e-01,
         2.78434545e-01,  1.91759899e-01, -2.46088222e-01,
         9.47210073e-01,  7.00283572e-02, -4.24034804e-01,
         7.75942862e-01,  3.51934016e-01,  1.83819816e-01,
         2.74857879e-01, -9.44027841e-01,  1.07133222e+00,
         7.11765170e-01,  7.98530936e-01,  3.93428892e-01,
        -6.34743631e-01,  7.05477782e-03,  1.19718358e-01,
         1.00800705e+00,  2.89274246e-01,  7.89648741e-02,
        -6.47306740e-01,  1.98634222e-01, -1.56902552e-0

## 向量搜索

In [40]:
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}

result = collection.search(embeddings, "vector", search_params, limit=3, output_fields=["pk" ,"source", "text"])

for hits in result:
    print(hits)
    for hit in hits:
        print(f"主键：{hit.entity.get('pk')}\n来源: {hit.entity.get('source')}\n文本: {hit.entity.get('text')}\n=============")        
        # print(f"hit: {hit}, 来源: {hit.entity.get('source')}\n=============")

["id: 446016573613050419, distance: 104.68807983398438, entity: {'pk': 446016573613050419, 'source': '员工报销问题自助手册.docx', 'text': '10、每刻报销的报销流程是什么？\\n\\n提单人线上提交单据，操作步骤详见《私人报销（每刻）操作流程》；\\n\\n线上流程到达“签收节点”，打印纸质单据交到财务室扫码签收；\\n\\n经办会计审核；\\n\\n线上流程到达出纳，出纳付款；\\n\\n流程结束。\\n\\n11、每刻报销该怎么操作？'}", "id: 446016573613050420, distance: 128.66665649414062, entity: {'pk': 446016573613050420, 'source': '员工报销问题自助手册.docx', 'text': '经办会计审核；\\n\\n线上流程到达出纳，出纳付款；\\n\\n流程结束。\\n\\n11、每刻报销该怎么操作？\\n\\n出差前需要创建出差申请单，预估各项消费金额，如果出差申请单忘记预估金额，则需要补上一张费用申请单并关联；出差中需要在手机企业微信的每刻报销，智能识票拍照，记录每一笔费用，并在我的发票验真并生成费用；出差后把生成的费用创建差旅报销单进行报销。\\n\\n12、一个出差单可以关联多个申请单吗？'}", "id: 446016573613050429, distance: 128.86740112304688, entity: {'pk': 446016573613050429, 'source': '员工报销问题自助手册.docx', 'text': '每刻报销，每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）。\\n\\n24、每刻报销一直没收到钱是怎么回事？\\n\\n如果系统显示已完成，可是没收到钱，则要检查自己的银行卡是不是填错，如果银行卡填错的话需要在每刻系统修改并且把正确的开户行、银行卡号邮件发给对应的会计和出纳。\\n\\n25、滴滴打车需要上传行程单吗？\\n\\n滴滴打车需要把发票和行程单一起上传并打印出来。'}"]
主键：446016573613050419
来源: 员工报销问题自助手册.docx
文本

In [65]:
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}

result = collection.search(embeddings, "vector", search_params, limit=3, output_fields=["source", "text"])
print("查询语句：", sentences)
for hit in result[0]:
    print(f"主键: {hit.id}\n来源: {hit.entity.get('source')}\n=============")
    # print(f"来源: {hit.entity.get('source')}\n=============")
    # print(f"来源: {hit.entity}\n=============")    

查询语句： ['每刻报销的报销流程是什么？']
主键: 446016573613050419
来源: 员工报销问题自助手册.docx
主键: 446016573613050420
来源: 员工报销问题自助手册.docx
主键: 446016573613050429
来源: 员工报销问题自助手册.docx


In [66]:
result[0]

["id: 446016573613050419, distance: 104.68807983398438, entity: {'text': '10、每刻报销的报销流程是什么？\\n\\n提单人线上提交单据，操作步骤详见《私人报销（每刻）操作流程》；\\n\\n线上流程到达“签收节点”，打印纸质单据交到财务室扫码签收；\\n\\n经办会计审核；\\n\\n线上流程到达出纳，出纳付款；\\n\\n流程结束。\\n\\n11、每刻报销该怎么操作？', 'source': '员工报销问题自助手册.docx'}", "id: 446016573613050420, distance: 128.66665649414062, entity: {'text': '经办会计审核；\\n\\n线上流程到达出纳，出纳付款；\\n\\n流程结束。\\n\\n11、每刻报销该怎么操作？\\n\\n出差前需要创建出差申请单，预估各项消费金额，如果出差申请单忘记预估金额，则需要补上一张费用申请单并关联；出差中需要在手机企业微信的每刻报销，智能识票拍照，记录每一笔费用，并在我的发票验真并生成费用；出差后把生成的费用创建差旅报销单进行报销。\\n\\n12、一个出差单可以关联多个申请单吗？', 'source': '员工报销问题自助手册.docx'}", "id: 446016573613050429, distance: 128.86740112304688, entity: {'text': '每刻报销，每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）。\\n\\n24、每刻报销一直没收到钱是怎么回事？\\n\\n如果系统显示已完成，可是没收到钱，则要检查自己的银行卡是不是填错，如果银行卡填错的话需要在每刻系统修改并且把正确的开户行、银行卡号邮件发给对应的会计和出纳。\\n\\n25、滴滴打车需要上传行程单吗？\\n\\n滴滴打车需要把发票和行程单一起上传并打印出来。', 'source': '员工报销问题自助手册.docx'}"]

## 标量查询

https://milvus.io/docs/boolean.md

仅支持匹配前缀

In [86]:
result1 = collection.query(expr="text like '每%'", limit=20, output_fields=["text", "source"])
for hits in result1:
    print(hits)
    # print(f"pk:{hits['pk']}\ntext: {hits['text']}\n ")


{'text': '每个月绑定公司银行卡自动扣款的，待回票后要提交销账单流程，其他的都要选择付款申请书；\n\n注：（惠州动力新能源公司需走单独的惠州动力新能源付款申请书和销账单）\n\n3、对公付款（OA）需要什么附件？\n\n对公付款需要上传双方盖章版合同、盖章版对账单（若核对内容较多，请同时上传EXCEL版对账单）、发票扫描件、特殊报销需提供请示等附件。\n\n4、对公付款（OA）上传发票注意事项是什么？', 'source': '员工报销问题自助手册.docx', 'pk': 446016573613050412}
{'text': '每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）\n\n9、为什么我的OA单据到了出纳一直没有付款？\n\nOA单据到了出纳，要进行第二次打印完整的流程，即：打印流程节点到出纳的报销单封面，到经办会计这边找到第一次交的单据，粘贴在最上面，经办会计才可以做台账交接给出纳付款。\n\n10、每刻报销的报销流程是什么？\n\n提单人线上提交单据，操作步骤详见《私人报销（每刻）操作流程》；', 'source': '员工报销问题自助手册.docx', 'pk': 446016573613050418}
{'text': '每刻报销，每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）。\n\n24、每刻报销一直没收到钱是怎么回事？\n\n如果系统显示已完成，可是没收到钱，则要检查自己的银行卡是不是填错，如果银行卡填错的话需要在每刻系统修改并且把正确的开户行、银行卡号邮件发给对应的会计和出纳。\n\n25、滴滴打车需要上传行程单吗？\n\n滴滴打车需要把发票和行程单一起上传并打印出来。', 'source': '员工报销问题自助手册.docx', 'pk': 446016573613050429}


In [87]:
result2 = collection.query(expr="text like '3%'", output_fields=["text", "source"])
for hits in result2:
    print(f"hit: {hits}")


hit: {'text': '34、公文请示在哪里写？\n\n在OA——公文管理——请示，请示会签到哪里，具体参考财务权限审批表。\n\n35、费用报销超时，显示无法提交单据怎么办？\n\n目前公司的报销制度规定，出差结束后两个月内必须完成线上提交与线下打印纸质单提交到财务处扫描签收。如果超时则无法提交，需要写请示，会签到会计与财务经理，然后提交IT申报，IT查验领导同意的请示后，打开相应的权限，方可提交。', 'source': '员工报销问题自助手册.docx', 'pk': 446016573613050434}
hit: {'text': '36、我的每刻报销单据显示在签收节点是怎么回事？\n\n签收节点，需要把纸质单据打印出来，贴好附件后，拿到光明财务室扫码签收，就签收后才会到经办会计审核。\n\n37、开公司的车出差，需要选择哪个费用类型报销？\n\n借用公司车辆，需要选择对应的公司车辆加油费、停车费等带有公司车辆字眼的费用类型，不可以用私车公用补贴，并且要选择对应的车牌号。', 'source': '员工报销问题自助手册.docx', 'pk': 446016573613050435}
hit: {'text': '38、原来的单子被经办会计驳回了，我把它作废后，需要重新打印新的报销单交过来财务办公室吗？\n\n旧的单子作废了，可是发票那些还在经办会计手上，需要报销人到驳回对应单据的费用会计处拿回发票等附件，将新的报销面单打印粘贴好发票附件，拿到财务室签收，重新审批。\n\n39、私车公用补贴需要单独提单吗？', 'source': '员工报销问题自助手册.docx', 'pk': 446016573613050436}
hit: {'text': '39、私车公用补贴需要单独提单吗？\n\n私车公用补贴需要单独提一张单子，部门总经理审核通过后由人资发放工资，选择：私车公用补贴单，过路费、高速费等其他费用依旧用差旅报销单。\n\n40、我的伙食补贴被扣减了，这是什么原因？\n\n在出差申请单的时候选了：有伙食，则系统会扣减；出差单应该选择：无伙食。\n\n超过30天的出差，选了伙食补贴，则系统也会扣减，正确的操作是在申请单的时候选择：长期出差补贴这个费用类型。', 'source': '员工报销问题自助手册.docx', 'pk': 446016573

## 混合查询

In [170]:
result3 = collection.search(embeddings,
                            "vector",
                            search_params,
                            limit=3,
                            expr="text like '每%'",
                            output_fields=["pk", "source", "text"])

for hits in result3:
    print(hits)
    # for hit in hits:
    #     print(f"hit: {hit}")

["id: 446016573613050429, distance: 128.86740112304688, entity: {'pk': 446016573613050429, 'source': '员工报销问题自助手册.docx', 'text': '每刻报销，每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）。\\n\\n24、每刻报销一直没收到钱是怎么回事？\\n\\n如果系统显示已完成，可是没收到钱，则要检查自己的银行卡是不是填错，如果银行卡填错的话需要在每刻系统修改并且把正确的开户行、银行卡号邮件发给对应的会计和出纳。\\n\\n25、滴滴打车需要上传行程单吗？\\n\\n滴滴打车需要把发票和行程单一起上传并打印出来。'}", "id: 446016573613050418, distance: 137.0202178955078, entity: {'pk': 446016573613050418, 'source': '员工报销问题自助手册.docx', 'text': '每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）\\n\\n9、为什么我的OA单据到了出纳一直没有付款？\\n\\nOA单据到了出纳，要进行第二次打印完整的流程，即：打印流程节点到出纳的报销单封面，到经办会计这边找到第一次交的单据，粘贴在最上面，经办会计才可以做台账交接给出纳付款。\\n\\n10、每刻报销的报销流程是什么？\\n\\n提单人线上提交单据，操作步骤详见《私人报销（每刻）操作流程》；'}", "id: 446016573613050412, distance: 162.8231201171875, entity: {'pk': 446016573613050412, 'source': '员工报销问题自助手册.docx', 'text': '每个月绑定公司银行卡自动扣款的，待回票后要提交销账单流程，其他的都要选择付款申请书；\\n\\n注：（惠州动力新能源公司需走单独的惠州动力新能源付款申请书和销账单）\\n\\n3、对公付款（OA）需要什么附件？\\n\\n对公付款需要上传双方盖章版合同、盖章版对账单（若核对内容较多，请同时上传EXCEL版对账单）、发票扫描件、特殊报销需提供请示等附件。\\n\\n4、对公付款（OA）上传发票注意事项是什

In [171]:
list(result3[0])

[id: 446016573613050429, distance: 128.86740112304688, entity: {'pk': 446016573613050429, 'source': '员工报销问题自助手册.docx', 'text': '每刻报销，每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）。\n\n24、每刻报销一直没收到钱是怎么回事？\n\n如果系统显示已完成，可是没收到钱，则要检查自己的银行卡是不是填错，如果银行卡填错的话需要在每刻系统修改并且把正确的开户行、银行卡号邮件发给对应的会计和出纳。\n\n25、滴滴打车需要上传行程单吗？\n\n滴滴打车需要把发票和行程单一起上传并打印出来。'},
 id: 446016573613050418, distance: 137.0202178955078, entity: {'pk': 446016573613050418, 'source': '员工报销问题自助手册.docx', 'text': '每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）\n\n9、为什么我的OA单据到了出纳一直没有付款？\n\nOA单据到了出纳，要进行第二次打印完整的流程，即：打印流程节点到出纳的报销单封面，到经办会计这边找到第一次交的单据，粘贴在最上面，经办会计才可以做台账交接给出纳付款。\n\n10、每刻报销的报销流程是什么？\n\n提单人线上提交单据，操作步骤详见《私人报销（每刻）操作流程》；'},
 id: 446016573613050412, distance: 162.8231201171875, entity: {'pk': 446016573613050412, 'source': '员工报销问题自助手册.docx', 'text': '每个月绑定公司银行卡自动扣款的，待回票后要提交销账单流程，其他的都要选择付款申请书；\n\n注：（惠州动力新能源公司需走单独的惠州动力新能源付款申请书和销账单）\n\n3、对公付款（OA）需要什么附件？\n\n对公付款需要上传双方盖章版合同、盖章版对账单（若核对内容较多，请同时上传EXCEL版对账单）、发票扫描件、特殊报销需提供请示等附件。\n\n4、对公付款（OA）上传发票注意事项是什么？'}]

In [172]:
result3[0][0].entity

id: 446016573613050429, distance: 128.86740112304688, entity: {'pk': 446016573613050429, 'source': '员工报销问题自助手册.docx', 'text': '每刻报销，每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）。\n\n24、每刻报销一直没收到钱是怎么回事？\n\n如果系统显示已完成，可是没收到钱，则要检查自己的银行卡是不是填错，如果银行卡填错的话需要在每刻系统修改并且把正确的开户行、银行卡号邮件发给对应的会计和出纳。\n\n25、滴滴打车需要上传行程单吗？\n\n滴滴打车需要把发票和行程单一起上传并打印出来。'}

## 查询结果处理

In [173]:
import re
import ast
import pandas as pd

data = []
for hit in result3[0]:
    s = str(hit.entity)
    entity_match = re.search(r'entity: (\{.*\})', s)
    if entity_match:
        # 提取出实体字典
        entity_str = entity_match.group(1)
        entity_dict = ast.literal_eval(entity_str)

        entity_dict.update({'distance': hit.distance})
        data.append(entity_dict)
    else:
        print("No entity found in the input string.")

df = pd.DataFrame(data)
df

,pk,source,text,distance
0,446016573613050429,员工报销问题自助手册.docx,每刻报销，每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）。\n\n24、每刻...,128.867401
1,446016573613050418,员工报销问题自助手册.docx,每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）\n\n9、为什么我的OA单据...,137.020218
2,446016573613050412,员工报销问题自助手册.docx,每个月绑定公司银行卡自动扣款的，待回票后要提交销账单流程，其他的都要选择付款申请书；\n\n...,162.823120


In [174]:
data1 = df.to_dict(orient='records')
data1

[{'pk': 446016573613050429,
  'source': '员工报销问题自助手册.docx',
  'text': '每刻报销，每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）。\n\n24、每刻报销一直没收到钱是怎么回事？\n\n如果系统显示已完成，可是没收到钱，则要检查自己的银行卡是不是填错，如果银行卡填错的话需要在每刻系统修改并且把正确的开户行、银行卡号邮件发给对应的会计和出纳。\n\n25、滴滴打车需要上传行程单吗？\n\n滴滴打车需要把发票和行程单一起上传并打印出来。',
  'distance': 128.86740112304688},
 {'pk': 446016573613050418,
  'source': '员工报销问题自助手册.docx',
  'text': '每周二、四付款，月底三个工作日不付款（除非特别紧急的付款单据）\n\n9、为什么我的OA单据到了出纳一直没有付款？\n\nOA单据到了出纳，要进行第二次打印完整的流程，即：打印流程节点到出纳的报销单封面，到经办会计这边找到第一次交的单据，粘贴在最上面，经办会计才可以做台账交接给出纳付款。\n\n10、每刻报销的报销流程是什么？\n\n提单人线上提交单据，操作步骤详见《私人报销（每刻）操作流程》；',
  'distance': 137.0202178955078},
 {'pk': 446016573613050412,
  'source': '员工报销问题自助手册.docx',
  'text': '每个月绑定公司银行卡自动扣款的，待回票后要提交销账单流程，其他的都要选择付款申请书；\n\n注：（惠州动力新能源公司需走单独的惠州动力新能源付款申请书和销账单）\n\n3、对公付款（OA）需要什么附件？\n\n对公付款需要上传双方盖章版合同、盖章版对账单（若核对内容较多，请同时上传EXCEL版对账单）、发票扫描件、特殊报销需提供请示等附件。\n\n4、对公付款（OA）上传发票注意事项是什么？',
  'distance': 162.8231201171875}]